# CNTK 202: Language Understanding with Recurrent Networks

This tutorial shows how to implement a recurrent network to process text,
for the [Air Travel Information Services](https://catalog.ldc.upenn.edu/LDC95S26) 
(ATIS) task of slot tagging (tag individual words to their respective classes, 
where the classes are provided as labels in the training data set).

There are 2 parts to this tutorial:
- Part 1: We will tag each word in a sequence to their corresponding label
- Part 2: We will classify a sequence to its corresponding intent.

We will start with a straight-forward (linear) embedding of the words followed by a recurrent LSTM to label each word in a sequence to the corresponding class. We will show how to classify each word token in a sequence to the corresponding class. This will then be extended to include neighboring words and run bidirectionally.

We will take the last state of the sequence and train a model that classifies the entire sequence to the corresponding class label (in this case the intent associated with the sequence).

The techniques you will practice are:
* model description by composing layer blocks, a convenient way to compose 
  networks/models without requiring the need to write formulas,
* creating your own layer block
* variables with different sequence lengths in the same network
* training the network

We assume that you are familiar with basics of deep learning, and these specific concepts:
* recurrent networks ([Wikipedia page](https://en.wikipedia.org/wiki/Recurrent_neural_network))
* text embedding ([Wikipedia page](https://en.wikipedia.org/wiki/Word_embedding))

## Prerequisites

We assume that you have already [installed CNTK](https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-CNTK-on-your-machine).
This tutorial requires CNTK V2. We strongly recommend to run this tutorial on a machine with
a capable CUDA-compatible GPU. Deep learning without GPUs is not fun.

_unchecked_

# CNTK 202: リカレントネットワークによる言語理解

このチュートリアルでは、テキストを処理するためのリカレントネットワークを実装する方法、スロットタグの[Air Travel インフォメーションサービス](https://catalog.ldc.upenn.edu/LDC95S26)(ATIS) タスクについて説明します (個々の単語をそれぞれのクラスにタグ付けし、クラスをトレーニングデータのラベルとして提供します)。セット)。

このチュートリアルには2つの部分があります:-第1部: 我々は、対応するラベルにシーケンス内の各単語をタグ付けします-パート 2: 我々は、対応する意図にシーケンスを分類します。

我々は、対応するクラスにシーケンス内の各単語にラベルを付けるために、再帰的な LSTM が続く単語のストレートフォワード (リニア) 埋め込みから始まります。シーケンス内の各単語トークンを対応するクラスに分類する方法について説明します。これは、隣接する単語を含むように拡張され、双を実行します。

シーケンスの最後の状態を取得し、シーケンス全体を対応するクラスラベル (この場合はシーケンスに関連付けられたインテント) に分類するモデルをトレーニングします。

あなたが実践するテクニックは次のとおりです:*レイヤーブロックを構成することによってモデルの説明、ネットワークを作成するための便利な方法/モデルを必要とせずに、*独自の層のブロックの変数を作成するための数式を記述するために*異なるシーケンスの長さ同じネットワーク*ネットワークのトレーニング

私たちは、あなたが深い学習の基礎に精通していると仮定し、これらの特定の概念:*リカレントネットワーク (*テキストの埋め込み ([ウィキペディアのページ](https://en.wikipedia.org/wiki/Word_embedding))

## 条件

既に[CNTK がインストールされ](https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-CNTK-on-your-machine)ていることを前提としています。
このチュートリアルでは、CNTK V2 が必要です。このチュートリアルは、CUDA 互換の GPU を搭載したマシンで実行することを強くお勧めします。gpu のない深い学習は楽しくありません。

## Data download

In this tutorial, we are going to use a (lightly preprocessed) version of the ATIS dataset. You can download the data automatically by running the cells below or by executing the manual instructions.

**Fallback manual instructions**
Download the ATIS [training](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Tutorials/SLUHandsOn/atis.train.ctf) 
and [test](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Tutorials/SLUHandsOn/atis.test.ctf) 
files and put them at the same folder as this notebook. If you want to see how the model is 
predicting on new sentences you will also need the vocabulary files for 
[queries](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Examples/LanguageUnderstanding/ATIS/BrainScript/query.wl) and
[slots](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Examples/LanguageUnderstanding/ATIS/BrainScript/slots.wl)

_unchecked_

## データのダウンロード

このチュートリアルでは、ATIS データセットの (軽く前処理された) バージョンを使用する予定です。以下のセルを実行するか、手動で指示を実行して、データを自動的にダウンロードできます。

**フォールバックの手動指示**
ATIS の[トレーニング](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Tutorials/SLUHandsOn/atis.train.ctf)と[テスト](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Tutorials/SLUHandsOn/atis.test.ctf)ファイルをダウンロードして、このノートブックと同じフォルダに配置します。新しい文章でモデルがどのように予測されているかを確認したい場合は、[クエリ](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Examples/LanguageUnderstanding/ATIS/BrainScript/query.wl)と[スロット](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Examples/LanguageUnderstanding/ATIS/BrainScript/slots.wl)のボキャブラリファイルも必要になります

In [1]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
import requests
import os

def download(url, filename):
    """ utility function to download a file """
    response = requests.get(url, stream=True)
    with open(filename, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

locations = ['Tutorials/SLUHandsOn', 'Examples/LanguageUnderstanding/ATIS/BrainScript']

data = {
  'train': { 'file': 'atis.train.ctf', 'location': 0 },
  'test': { 'file': 'atis.test.ctf', 'location': 0 },
  'query': { 'file': 'query.wl', 'location': 1 },
  'slots': { 'file': 'slots.wl', 'location': 1 },
  'intent': { 'file': 'intent.wl', 'location': 1 }  
}

for item in data.values():
    location = locations[item['location']]
    path = os.path.join('..', location, item['file'])
    if os.path.exists(path):
        print("Reusing locally cached:", item['file'])
        # Update path
        item['file'] = path
    elif os.path.exists(item['file']):
        print("Reusing locally cached:", item['file'])
    else:
        print("Starting download:", item['file'])
        url = "https://github.com/Microsoft/CNTK/blob/release/2.3.1/%s/%s?raw=true"%(location, item['file'])
        download(url, item['file'])
        print("Download completed")


Reusing locally cached: query.wl
Reusing locally cached: intent.wl
Reusing locally cached: atis.test.ctf
Reusing locally cached: atis.train.ctf
Reusing locally cached: slots.wl


**Importing libraries**: CNTK, math and numpy 

CNTK's Python module contains several submodules like `io`, `learner`, and `layers`. We also use NumPy in some cases since the results returned by CNTK work like NumPy arrays.

_unchecked_

**ライブラリのインポート**: CNTK、数学、numpy

CNTK の Python モジュールには、 `io` 、 `learner` 、 `layers` のようないくつかのサブが含まれています。また、NumPy 配列のように CNTK によって返される結果から、いくつかのケースで NumPy を使用します。

In [2]:
import math
import numpy as np

import cntk as C
import cntk.tests.test_utils
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components

## Task overview: Slot tagging

The task we want to approach in this tutorial is slot tagging.
We use the [ATIS corpus](https://catalog.ldc.upenn.edu/LDC95S26).
ATIS contains human-computer queries from the domain of Air Travel Information Services,
and our task will be to annotate (tag) each word of a query whether it belongs to a
specific item of information (slot), and which one.

The data in your working folder has already been converted into the "CNTK Text Format."
Let us look at an example from the test-set file `atis.test.ctf`:

    19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O

This file has 7 columns:

* a sequence id (19). There are 11 entries with this sequence id. This means that sequence 19 consists
of 11 tokens;
* column `S0`, which contains numeric word indices; the input data is encoded in one-hot vectors.  There are 943 words in the vocabulary, so each word is a 943 element vector of all 0 with a 1 at a vector index chosen to represent that word.  For example the word "from" is represented with a 1 at index 444 and zero everywhere else in the vector. The word "monday" is represented with a 1 at index 601 and zero everywhere else in the vector.
* a comment column denoted by `#`, to allow a human reader to know what the numeric word index stands for;
Comment columns are ignored by the system. `BOS` and `EOS` are special words
to denote beginning and end of sentence, respectively;
* column `S1` is an intent label, which we will use in the second part of the tutorial;
* another comment column that shows the human-readable label of the numeric intent index;
* column `S2` is the slot label, represented as a numeric index; and
* another comment column that shows the human-readable label of the numeric label index.

The task of the neural network is to look at the query (column `S0`) and predict the
slot label (column `S2`).
As you can see, each word in the input gets assigned either an empty label `O`
or a slot label that begins with `B-` for the first word, and with `I-` for any
additional consecutive word that belongs to the same slot.

### Model Creation

The model we will use is a recurrent model consisting of an embedding layer,
a recurrent LSTM cell, and a dense layer to compute the posterior probabilities:


    slot label   "O"        "O"        "O"        "O"  "B-fromloc.city_name"
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Dense |  | Dense |  | Dense |  | Dense |  | Dense |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"

Or, as a CNTK network description. Please have a quick look and match it with the description above:
(descriptions of these functions can be found at: [the layers reference](http://cntk.ai/pythondocs/layerref.html))


_unchecked_

## タスクの概要: スロットのタグ付け

このチュートリアルでは、アプローチしたいタスクはスロットタグです。
我々は、 [ATIS コーパス](https://catalog.ldc.upenn.edu/LDC95S26)を使用します。
ATIS は、航空旅行情報サービスのドメインから人間のコンピュータのクエリが含まれており、我々のタスクは、それが情報 (スロット) の特定の項目に属しているかどうか、クエリの各単語 (タグ) に注釈を付けることになりますどの1。

作業フォルダ内のデータは、既に "CNTK テキスト形式" に変換されています。
テストセットファイル `atis.test.ctf` の例を見てみましょう。

    `19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O
    `

このファイルには7列があります。

- シーケンス id (19)。このシーケンス id のエントリは11個あります。つまり、シーケンス19は11のトークンで構成されます。

- 数値単語のインデックスを含む列 `S0` 。入力データは、1つのホットベクタでエンコードされます。 語彙には943の単語があるので、各単語は、その単語を表すために選択されたベクトルインデックスで1を持つすべての0の943要素ベクトルです。 たとえば、"from" という単語は、ベクター内の他の場所でインデックス444と0で1で表されます。"月曜日" という単語は、ベクター内の他の場所でインデックス601と0で1で表されます。

- `#`によって示されたコメント欄は、人間の読者が数字単語索引が何のために立つかを知ることを可能にします。
コメント列はシステムによって無視されます。`BOS`と `EOS` は、それぞれ、文の始まりと終わりを表す特殊な単語です。

- 列 `S1` は、チュートリアルの2番目の部分で使用するインテントラベルです。

- 数値インテントインデックスの人間が判読できるラベルを示す別のコメント列。

- 列 `S2` は、数値インデックスとして表されるスロットラベルです。

- 数値ラベルインデックスの人間が判読できるラベルを示す別のコメント列。

ニューラルネットワークのタスクは、クエリ (列 `S0` ) を見て、スロットラベル (列 `S2` ) を予測することです。
ご覧のように、入力内の各単語は、最初の単語の `B-` で始まる空のラベル `O` またはスロットラベルのいずれかに割り当てられ、同じスロットに属する追加の連続した単語に対して `I-` を使用します。

### モデルの作成

我々が使用するモデルは、後方の確率を計算するために、埋め込み層、再発 LSTM 細胞、および密な層から成る再発モデルです。

    `slot label   "O"        "O"        "O"        "O"  "B-fromloc.city_name"
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Dense |  | Dense |  | Dense |  | Dense |  | Dense |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"
    `

または、CNTK ネットワークの説明として。してください簡単に見て、上記の説明と一致する: (これらの関数の説明で見つけることができます:[レイヤー参照](http://cntk.ai/pythondocs/layerref.html))

In [3]:
# number of words in vocab, slot labels, and intent labels
vocab_size = 943 ; num_labels = 129 ; num_intents = 26    

# model dimensions
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 150
hidden_dim = 300

# Create the containers for input feature (x) and the label (y)
x = C.sequence.input_variable(vocab_size)
y = C.sequence.input_variable(num_labels)

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels, name='classify')
        ])

Now we are ready to create a model and inspect it. 

The model attributes are fully accessible from Python. The first layer named `embed` is an Embedding layer. Here we use the CNTK default, which is linear embedding. It is a simple matrix with dimension (input word encoding x output projected dimension). You can access its parameter `E` (where the embeddings are stored) like any other attribute of a Python object. Its shape contains a `-1` which indicates that this parameter (with input dimension) is not fully specified yet, while the output dimension is set to `emb_dim` ( = 150 in this tutorial). 

Additionally, we also inspect the value of the bias vector in the `Dense` layer named `classify`. The `Dense` layer is a fundamental compositional unit of a Multi-Layer Perceptron (as introduced in CNTK 103C tutorial). The `Dense` layer has both `weight` and `bias` parameters, one each per `Dense` layer. Bias terms are by default initialized to 0 (but there is a way to change that if you need). As you create the model, one should name the layer component and then access the parameters as shown here. 

**Suggested task**: What should be the expected dimension of the `weight` matrix from the layer named `classify`? Try printing the weight matrix of the `classify` layer? Does it match with your expected size?

_unchecked_

今は、モデルを作成し、それを検査する準備が整いました。

モデル属性は Python から完全にアクセスできます。`embed`という名前の最初のレイヤは、埋め込みレイヤです。ここでは、線形埋め込みである CNTK デフォルトを使用します。これは、ディメンション (入力ワードエンコード x 出力投影寸法) を持つ単純な行列です。Python オブジェクトの他の属性と同様に、パラメータ `E` (込みが格納されている場所) にアクセスできます。その形状には、このパラメータ (入力寸法を含む) が完全に指定されていないことを示す `-1` が含まれていますが、出力ディメンションは `emb_dim` (このチュートリアルでは 150) に設定されています。

また、 `classify` という `Dense` 層のバイアスベクトルの値も検査します。`Dense`層は、マルチレイヤパーセプトロンの基本構成単位 (CNTK 103C チュートリアルで紹介) です。`Dense`レイヤには、 `weight` と `bias` パラメータの両方があり、 `Dense` レイヤごとに1つずつあります。バイアス条件はデフォルトで0に初期化されます (ただし、必要に応じて変更する方法があります)。モデルを作成する際には、レイヤコンポーネントに名前を付け、次に示すようにパラメータにアクセスする必要があります。

推奨される**タスク**: `weight` という名前のレイヤーからマトリックスの予想されるディメンションを指定してください `classify` 。`classify`レイヤーのウェイトマトリックスを印刷してみますか?それはあなたの予想サイズと一致していますか?

In [4]:
# peek
z = create_model()
print(z.embed.E.shape)
print(z.classify.b.value)

(-1, 150)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.]


In our case we have input as one-hot encoded vector of length 943 and the output dimension `emb_dim` is set to 150. In the code below we pass the input variable `x` to our model `z`. This binds the model with input data of known shape. In this case, the input shape will be the size of the input vocabulary. With this modification, the parameter returned by the embed layer is completely specified (943, 150). **Note**: You can initialize the Embedding matrix with pre-computed vectors using [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) or [GloVe](https://en.wikipedia.org/wiki/GloVe_%28machine_learning%29).

_unchecked_

我々のケースでは、長さ943の1ホットエンコードされたベクトルとして入力しており、出力寸法 `emb_dim` は150に設定しています。以下のコードでは、入力変数 `x` をモデル `z` に渡しています。これは既知の図形の入力データとモデルをバインドします。この場合、入力図形は入力ボキャブラリのサイズになります。この変更により、埋め込みレイヤによって返されるパラメータは完全に指定されます (943, 150)。**注**: 埋め込み行列は、 [Word2Vec](https://en.wikipedia.org/wiki/Word2vec)または[グローブ](https://en.wikipedia.org/wiki/GloVe_%28machine_learning%29)を使用して事前計算されたベクタで初期化できます。

In [5]:
# Pass an input and check the dimension
z = create_model()
print(z(x).embed.E.shape)

(943, 150)


To train and test a model in CNTK, we need to create a model and specify how to read data and perform training and testing. 

In order to train we need to specify:

* how to read the data 
* the model function, its inputs, and outputs
* hyper-parameters for the learner such as the learning rate

[comment]: <> (For testing ...)

## Data Reading

We already looked at the data.
But how do you generate this format?
For reading text, this tutorial uses the `CNTKTextFormatReader`. It expects the input data to be
in a specific format, as described [here](https://docs.microsoft.com/en-us/cognitive-toolkit/Brainscript-CNTKTextFormat-Reader).

For this tutorial, we created the corpora by two steps:
* convert the raw data into a plain text file that contains of TAB-separated columns of space-separated text. For example:

  ```
  BOS show flights from burbank to st. louis on monday EOS (TAB) flight (TAB) O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.day_name O
  ```

  This is meant to be compatible with the output of the `paste` command.
* convert it to CNTK Text Format (CTF) with the following command:

  ```
  python [CNTK root]/Scripts/txt2ctf.py --map query.wl intent.wl slots.wl --annotated True --input atis.test.txt --output atis.test.ctf
  ```
  where the three `.wl` files give the vocabulary as plain text files, one word per line.

In these CTF files, our columns are labeled `S0`, `S1`, and `S2`.
These are connected to the actual network inputs by the corresponding lines in the reader definition:

_unchecked_

CNTK でモデルをトレーニングしてテストするには、モデルを作成し、データの読み取りとトレーニングとテストの実行方法を指定する必要があります。

我々は指定する必要が訓練するために:

- データの読み方

- モデル関数、その入力、および出力

- 学習速度などの学習者のためのハイパーパラメータ

## データの読み取り

我々はすでにデータを見た。
しかし、どのようにこの形式を生成するのですか?
テキストを読み取る場合、このチュートリアルでは `CNTKTextFormatReader` を使用します。入力データは、[ここ](https://docs.microsoft.com/en-us/cognitive-toolkit/Brainscript-CNTKTextFormat-Reader)で説明したように、特定の形式になることを想定しています。

このチュートリアルでは、次の2つの手順によってコーパスを作成しました: * raw データを、空白で区切られたテキストのタブ区切りの列を含むプレーンテキストファイルに変換します。例えば：

`BOS show flights from burbank to st. louis on monday EOS (TAB) flight (TAB) O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.day_name O`

これは、 `paste` コマンドの出力と互換性があることを意味します。 * 次のコマンドを使用して、CNTK テキスト形式 (CTF) に変換します。

`python [CNTK root]/Scripts/txt2ctf.py --map query.wl intent.wl slots.wl --annotated True --input atis.test.txt --output atis.test.ctf`ここで、3つの `.wl` ファイルは、プレーンテキストファイルとして語彙を与える、1行あたりの単語。

これらの CTF ファイルでは、列に `S0` 、 `S1` 、および `S2` というラベルが付けられています。
これらは、リーダー定義の対応する行によって、実際のネットワーク入力に接続されます。

In [6]:
def create_reader(path, is_training):
    return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
         query         = C.io.StreamDef(field='S0', shape=vocab_size,  is_sparse=True),
         intent        = C.io.StreamDef(field='S1', shape=num_intents, is_sparse=True),  
         slot_labels   = C.io.StreamDef(field='S2', shape=num_labels,  is_sparse=True)
     )), randomize=is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)

In [7]:
# peek
reader = create_reader(data['train']['file'], is_training=True)
reader.streams.keys()

dict_keys(['query', 'slot_labels', 'intent'])

## Training

We also must define the training criterion (loss function), and also an error metric to track. In most tutorials, we know the input dimensions and the corresponding labels. We directly create the loss and the error functions. In this tutorial we will do the same. However, we take a brief detour and learn about placeholders. This concept would be useful for Task 3. 

**Learning note**: Introduction to `placeholder`: Remember that the code we have been writing is not actually executing any heavy computation it is just specifying the function we want to compute on data during training/testing. And in the same way that it is convenient to have names for arguments when you write a regular function in a programming language, it is convenient to have placeholders that refer to arguments (or local computations that need to be reused). Eventually, some other code will replace these placeholders with other known quantities in the same way that in a programming language the function will be called with concrete values bound to its arguments. 

Specifically, the input variables you have created above `x = C.sequence.input_variable(vocab_size)` holds data pre-defined by `vocab_size`. In the case where such instantiations are challenging or not possible, using `placeholder` is a logical choice. Having the `placeholder` only allows you to defer the specification of the argument at a later time when you may have the data.

Here is an example below that illustrates the use of `placeholder`.

_unchecked_

## トレーニング

また、訓練基準 (損失機能) を定義する必要があり、また、追跡するためのエラーメトリック。ほとんどのチュートリアルでは、入力寸法と対応するラベルを知っています。我々は、直接損失とエラー機能を作成します。このチュートリアルでは、同じことを行います。しかし、我々は簡単な回り道をし、プレースホルダについて学ぶ。この概念は、タスク3に役立ちます。

**学習注**: の概要: ここで記述しているコードは、実際には重い計算を実行しているわけではないので、 `placeholder` トレーニング/テスト中にデータを計算したい関数を指定するだけです。また、プログラミング言語で通常の関数を記述するときに引数に名前を付けるのが便利なのと同じ方法で、引数 (または再利用する必要があるローカル計算) を参照するプレースホルダがあると便利です。最終的に、他のコードでは、プログラミング言語で関数が引数にバインドされた具象値で呼び出されるのと同じ方法で、これらのプレースホルダを他の既知の数量と置き換えます。

具体的には、 `x = C.sequence.input_variable(vocab_size)` 上に作成した入力変数は、 `vocab_size` によってあらかじめ定義されたデータを保持します。このようなインスタンス化が困難であるか、不可能である場合、 `placeholder` を使用することは論理的な選択です。`placeholder`を使用すると、データがある可能性があるときに、後で引数の指定を遅らせることができます。

以下に、 `placeholder` の使用例を示します。

In [8]:
def create_criterion_function(model):
    labels = C.placeholder(name='labels')
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    return C.combine ([ce, errs]) # (features, labels) -> (loss, metric)

criterion = create_criterion_function(create_model())
criterion.replace_placeholders({criterion.placeholders[0]: C.sequence.input_variable(num_labels)})

Composite(Combine): Input('Input2300', [#, *], [129]), Placeholder('labels', [???], [???]) -> Output('Block2270_Output_0', [#, *], [1]), Output('Block2290_Output_0', [#, *], [])

While the cell above works well when one has input parameters defined at network creation, it compromises readability. Hence we prefer creating functions as shown below

_unchecked_

上記のセルは、ネットワークの作成時に入力パラメータが定義されている場合にうまく機能しますが、読みやすさが損なわれます。したがって、我々は以下のように関数を作成することを好む

In [9]:
def create_criterion_function_preferred(model, labels):
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    return ce, errs # (model, labels) -> (loss, error metric)

In [10]:
def train(reader, model_func, max_epochs=10, task='slot_tagging'):
    
    # Instantiate the model function; x is the input (feature) variable 
    model = model_func(x)
    
    # Instantiate the loss and error function
    loss, label_error = create_criterion_function_preferred(model, y)

    # training config
    epoch_size = 18000        # 18000 samples is half the dataset size 
    minibatch_size = 70
    
    # LR schedule over epochs 
    # In CNTK, an epoch is how often we get out of the minibatch loop to
    # do other stuff (e.g. checkpointing, adjust learning rate, etc.)
    lr_per_sample = [3e-4]*4+[1.5e-4]
    lr_per_minibatch = [lr * minibatch_size for lr in lr_per_sample]
    lr_schedule = C.learning_parameter_schedule(lr_per_minibatch, epoch_size=epoch_size)
    
    # Momentum schedule
    momentums = C.momentum_schedule(0.9048374180359595, minibatch_size=minibatch_size)
    
    # We use a the Adam optimizer which is known to work well on this dataset
    # Feel free to try other optimizers from 
    # https://www.cntk.ai/pythondocs/cntk.learner.html#module-cntk.learner
    learner = C.adam(parameters=model.parameters,
                     lr=lr_schedule,
                     momentum=momentums,
                     gradient_clipping_threshold_per_sample=15, 
                     gradient_clipping_with_truncation=True)

    # Setup the progress updater
    progress_printer = C.logging.ProgressPrinter(tag='Training', num_epochs=max_epochs)
    
    # Uncomment below for more detailed logging
    #progress_printer = ProgressPrinter(freq=100, first=10, tag='Training', num_epochs=max_epochs) 

    # Instantiate the trainer
    trainer = C.Trainer(model, (loss, label_error), learner, progress_printer)

    # process minibatches and perform model training
    C.logging.log_number_of_parameters(model)
    
    # Assign the data fields to be read from the input
    if task == 'slot_tagging':
        data_map={x: reader.streams.query, y: reader.streams.slot_labels}
    else:
        data_map={x: reader.streams.query, y: reader.streams.intent} 

    t = 0
    for epoch in range(max_epochs):         # loop over epochs
        epoch_end = (epoch+1) * epoch_size
        while t < epoch_end:                # loop over minibatches on the epoch
            data = reader.next_minibatch(minibatch_size, input_map= data_map)  # fetch minibatch
            trainer.train_minibatch(data)               # update model with it
            t += data[y].num_samples                    # samples so far
        trainer.summarize_training_progress()

**Run the trainer**

You can find the complete recipe below.

_unchecked_

**トレーナーを実行する**

あなたは、以下の完全なレシピを見つけることができます。

In [11]:
def do_train():
    global z
    z = create_model()
    reader = create_reader(data['train']['file'], is_training=True)
    train(reader, z)
do_train()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.740198 * 18010, metric = 28.02% * 18010 6.466s (2785.3 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.665177 * 18051, metric = 14.30% * 18051 5.238s (3446.2 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.526256 * 17941, metric = 11.34% * 17941 5.198s (3451.5 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.395405 * 18059, metric = 8.22% * 18059 5.329s (3388.8 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.293512 * 17957, metric = 6.20% * 17957 5.106s (3516.8 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.264932 * 18021, metric = 5.73% * 18021 5.335s (3377.9 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.217258 * 17980, metric = 4.69% * 17980 5.248s (3426.1 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.209614 * 18025, metric =

This shows how learning proceeds over epochs (passes through the data).
For example, after four epochs, the loss, which is the cross-entropy criterion, 
has reduced significantly as measured on the ~18000 samples of this epoch,
and the same with the error rate on those same 18000 training samples.

The epoch size is the number of samples--counted as *word tokens*, not sentences--to
process between model checkpoints.

Once the training has completed (a little less than 2 minutes on a Titan-X or a Surface Book),
you will see an output like this
```
Finished Epoch[10 of 10]: [Training] loss = 0.157653 * 18039, metric = 3.41% * 18039
```
which is the loss (cross entropy) and the metric (classification error) averaged over the final epoch.

On a CPU-only machine, it can be 4 or more times slower. You can try setting
```python
emb_dim    = 50 
hidden_dim = 100
```
to reduce the time it takes to run on a CPU, but the model will not fit as well as when the 
hidden and embedding dimension are larger. 

_unchecked_

これは、エポック (データを通過) の学習がどのように進むかを示しています。
例えば、4つのエポックの後、クロスエントロピー基準である損失は、このエポックの ~ 18000 サンプルで測定されたように大幅に減少し、同じ18000トレーニングサンプルのエラー率と同じになります。

エポックサイズは、モデルのチェックポイント間で処理するために、*ワードトークン*として数えられるサンプルの数です (文章ではありません)。

トレーニングが完了したら (タイタン-X または表面の本で少し2分未満)、この `Finished Epoch[10 of 10]: [Training] loss = 0.157653 * 18039, metric = 3.41% * 18039` のような出力が表示されます損失 (クロスエントロピー) とメトリック (分類エラー) は、最後のエポックの平均。

CPU 専用マシンでは、4回以上の時間がかかります。`python
emb_dim    = 50 
hidden_dim = 100`を設定して CPU での実行にかかる時間を短縮することはできますが、非表示の埋め込みディメンションのサイズが大きい場合と同様にモデルは適合しません。

### Evaluating the model

Like the train() function, we also define a function to measure accuracy on a test set by computing the error over multiple minibatches of test data. For evaluating on a small sample read from a file, you can set a minibatch size reflecting the sample size and run the test_minibatch on that instance of data. To see how to evaluate a single sequence, we provide an instance later in the tutorial. 

_unchecked_

### モデルの評価

列車 () 関数と同様に、テストデータの複数の minibatches にわたってエラーを計算することによって、テストセットの精度を測定する関数も定義します。ファイルから読み取られた小さなサンプルを評価する場合は、サンプルサイズを反映した minibatch サイズを設定し、そのデータのインスタンスで test_minibatch を実行することができます。1つのシーケンスを評価する方法を確認するには、後でチュートリアルでインスタンスを提供します。

In [12]:
def evaluate(reader, model_func, task='slot_tagging'):
    
    # Instantiate the model function; x is the input (feature) variable 
    model = model_func(x)
    
    # Create the loss and error functions
    loss, label_error = create_criterion_function_preferred(model, y)

    # process minibatches and perform evaluation
    progress_printer = C.logging.ProgressPrinter(tag='Evaluation', num_epochs=0)
    
    # Assign the data fields to be read from the input
    if task == 'slot_tagging':
        data_map={x: reader.streams.query, y: reader.streams.slot_labels}
    else:
        data_map={x: reader.streams.query, y: reader.streams.intent} 

    while True:
        minibatch_size = 500
        data = reader.next_minibatch(minibatch_size, input_map= data_map)  # fetch minibatch
        if not data:                                 # until we hit the end
            break

        evaluator = C.eval.Evaluator(loss, progress_printer)
        evaluator.test_minibatch(data)
     
    evaluator.summarize_test_progress()

Now we can measure the model accuracy by going through all the examples in the test set and using the ``C.eval.Evaluator`` method. 

_unchecked_

ここでは、テストセット内のすべての例と `C.eval.Evaluator` メソッドを使用して、モデルの精度を測定できます。

In [13]:
def do_test():
    reader = create_reader(data['test']['file'], is_training=False)
    evaluate(reader, z)
do_test()
z.classify.b.value

Finished Evaluation [1]: Minibatch[1-23]: metric = 0.34% * 10984;


array([ -1.86572317e-02,  -8.51036515e-03,   1.38878925e-02,
        -1.95176266e-02,  -2.78977025e-03,  -1.23168388e-02,
        -6.16775267e-03,  -1.48158008e-02,  -5.82036236e-03,
        -2.99133137e-02,  -1.39552690e-02,  -2.11144108e-02,
        -1.13499342e-02,  -1.25011550e-02,  -8.19404377e-04,
        -5.26463473e-03,  -2.67275460e-02,  -1.80706571e-04,
        -3.97865893e-03,  -2.99989916e-02,  -1.00385472e-02,
        -6.81575621e-03,  -2.65348833e-02,  -2.01367699e-02,
        -2.63106022e-02,   4.22888156e-03,   5.74267423e-03,
        -1.56373512e-02,   7.71288527e-04,  -2.11508083e-03,
        -9.64712165e-03,  -1.98591035e-02,  -1.32136559e-02,
         7.97899254e-03,  -1.76088810e-02,   9.19441786e-03,
         1.30802142e-02,  -3.85359419e-03,   1.86733739e-03,
        -5.96518070e-03,  -3.07163727e-02,  -3.04672867e-03,
        -3.46868881e-04,  -1.29565294e-03,  -4.47260169e-03,
        -1.29292896e-02,  -1.05356863e-02,  -9.16024856e-03,
         6.08767197e-03,

The following block of code illustrates how to evaluate a single sequence. Additionally we show how one can pass in the information using NumPy arrays.

_unchecked_

次のコードブロックは、1つのシーケンスを評価する方法を示しています。さらに、NumPy 配列を使用して情報をどのように渡すことができるかを示します。

In [14]:
# load dictionaries
query_wl = [line.rstrip('\n') for line in open(data['query']['file'])]
slots_wl = [line.rstrip('\n') for line in open(data['slots']['file'])]
query_dict = {query_wl[i]:i for i in range(len(query_wl))}
slots_dict = {slots_wl[i]:i for i in range(len(slots_wl))}

# let's run a sequence through
seq = 'BOS flights from new york to seattle EOS'
w = [query_dict[w] for w in seq.split()] # convert to word indices
print(w)
onehot = np.zeros([len(w),len(query_dict)], np.float32)
for t in range(len(w)):
    onehot[t,w[t]] = 1

#x = C.sequence.input_variable(vocab_size)
pred = z(x).eval({x:[onehot]})[0]
print(pred.shape)
best = np.argmax(pred,axis=1)
print(best)
list(zip(seq.split(),[slots_wl[s] for s in best]))

[178, 429, 444, 619, 937, 851, 752, 179]
(8, 129)
[128 128 128  48 110 128  78 128]


[('BOS', 'O'),
 ('flights', 'O'),
 ('from', 'O'),
 ('new', 'B-fromloc.city_name'),
 ('york', 'I-fromloc.city_name'),
 ('to', 'O'),
 ('seattle', 'B-toloc.city_name'),
 ('EOS', 'O')]

## Modifying the Model

In the following, you will be given tasks to practice modifying CNTK configurations.
The solutions are given at the end of this document... but please try without!

**A Word About [`Sequential()`](https://www.cntk.ai/pythondocs/layerref.html#sequential)**

Before jumping to the tasks, let's have a look again at the model we just ran.
The model is described in what we call *function-composition style*.
```python
        Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])
```
You may be familiar with the "sequential" notation from other neural-network toolkits.
If not, [`Sequential()`](https://www.cntk.ai/pythondocs/layerref.html#sequential) is a powerful operation that,
in a nutshell, allows to compactly express a very common situation in neural networks
where an input is processed by propagating it through a progression of layers.
`Sequential()` takes an list of functions as its argument,
and returns a *new* function that invokes these functions in order,
each time passing the output of one to the next.
For example,
```python
	FGH = Sequential ([F,G,H])
    y = FGH (x)
```
means the same as
```
    y = H(G(F(x))) 
```
This is known as ["function composition"](https://en.wikipedia.org/wiki/Function_composition),
and is especially convenient for expressing neural networks, which often have this form:

         +-------+   +-------+   +-------+
    x -->|   F   |-->|   G   |-->|   H   |--> y
         +-------+   +-------+   +-------+

Coming back to our model at hand, the `Sequential` expression simply
says that our model has this form:

         +-----------+   +----------------+   +------------+
    x -->| Embedding |-->| Recurrent LSTM |-->| DenseLayer |--> y
         +-----------+   +----------------+   +------------+

_unchecked_

## モデルの変更

以下では、CNTK 構成の変更を練習するためのタスクが与えられます。
ソリューションは、このドキュメントの最後に与えられています.

**について一言**

タスクにジャンプする前に、我々はちょうど走ったモデルでもう一度見てみましょう。
このモデルについては、「*関数-コンポジションスタイル*」と呼びます。
`python
        Sequential([
            Embedding(emb_dim),
            Recurrence(LSTM(hidden_dim), go_backwards=False),
            Dense(num_labels)
        ])`は、他のニューラルネットワークツールキットの "シーケンシャル" 表記に精通している可能性があります。
そうでない場合、 `Sequential()` は、簡単に言えば、入力がレイヤーの進行を通じてそれを伝播することによって処理されるニューラルネットワークの非常に一般的な状況をコンパクトに表現することができる強力な操作です。 `Sequential()` は引数として関数のリストを取り、
、次のいずれかの出力を渡すたびに、これらの関数を順番に呼び出す*new*関数を返します。
例えば、は、 `python
    FGH = Sequential ([F,G,H])
    y = FGH (x)` これが「関数構成」として知られていると同じ意味であり、 `y = H(G(F(x)))` 特にこの形式を持つニューラルネットワークを表現するのに便利です。 [None](https://en.wikipedia.org/wiki/Function_composition)

    `     +-------+   +-------+   +-------+
    x -->|   F   |-->|   G   |-->|   H   |--> y
         +-------+   +-------+   +-------+
    `

手元に戻って私たちのモデルに来て、 `Sequential` 式は、単に私たちのモデルは、このフォームを持っていると言う:

    `     +-----------+   +----------------+   +------------+
    x -->| Embedding |-->| Recurrent LSTM |-->| DenseLayer |--> y
         +-----------+   +----------------+   +------------+
    `

### Task 1: Add Batch Normalization

We now want to add new layers to the model, specifically batch normalization.

Batch normalization is a popular technique for speeding up convergence.
It is often used for image-processing setups. But could it work for recurrent models, too?

> Note: training with Batch Normalization is currently only supported on GPU.

So your task will be to insert batch-normalization layers before and after the recurrent LSTM layer.
If you have completed the [hands-on labs on image processing](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Tutorials/CNTK_201B_CIFAR-10_ImageHandsOn.ipynb),
you may remember that the [batch-normalization layer](https://www.cntk.ai/pythondocs/layerref.html#batchnormalization-layernormalization-stabilizer) has this form:
```
    BatchNormalization()
```
So please go ahead and modify the configuration and see what happens.

If everything went right, you will notice improved convergence speed (`loss` and `metric`)
compared to the previous configuration.

_unchecked_

### タスク 1: バッチ正規化の追加

ここでは、モデルに新しいレイヤーを追加し、特にバッチ正規化を行いたいと思っています。

バッチ正規化は、収束を高速化するための一般的な手法です。
これは、イメージ処理のセットアップによく使用されます。しかし、それも再発モデルのために働くことができる?

>注: バッチ正規化を使用したトレーニングは、現在 GPU でのみサポートされています。

そこで、あなたのタスクは、反復 LSTM 層の前後にバッチ正規化レイヤーを挿入することになります。
[画像処理に関するハンズオンラボ](https://github.com/Microsoft/CNTK/blob/release/2.3.1/Tutorials/CNTK_201B_CIFAR-10_ImageHandsOn.ipynb)が完了している場合は、[バッチ正規化レイヤ](https://www.cntk.ai/pythondocs/layerref.html#batchnormalization-layernormalization-stabilizer)にこのフォームがあることに注意して `BatchNormalization()` ください。ので、先に行って、設定を変更し、何が起こるかを参照してください。

すべてが正しかった場合は、以前の構成と比較して、収束速度 ( `loss` および `metric` ) が改善されます。

In [15]:
# Your task: Add batch normalization
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])

# Enable these when done:
#do_train()
#do_test()

### Task 2: Add a Lookahead 

Our recurrent model suffers from a structural deficit:
Since the recurrence runs from left to right, the decision for a slot label
has no information about upcoming words. The model is a bit lopsided.
Your task will be to modify the model such that
the input to the recurrence consists not only of the current word, but also of the next one
(lookahead).

Your solution should be in function-composition style.
Hence, you will need to write a Python function that does the following:

* takes no input arguments
* creates a placeholder (sequence) variable
* computes the "next value" in this sequence using the `sequence.future_value()` operation and
* concatenates the current and the next value into a vector of twice the embedding dimension using `splice()`

and then insert this function into `Sequential()`'s list right after the embedding layer.

_unchecked_

### タスク 2: 先読みを追加する

私たちの再発モデルは、構造的な赤字に苦しむ: 再発は左から右に実行されるため、スロットラベルの決定は、今後の単語についての情報を持っていません。モデルは少しなです。
あなたのタスクは、現在の単語だけでなく、次の1つ (先読み) の入力が繰り返されるようにモデルを変更することです。

ソリューションは、関数合成スタイルにする必要があります。
したがって、次のような Python 関数を記述する必要があります。

- 入力引数を受け取りません

- プレースホルダ (シーケンス) 変数を作成します。

- `sequence.future_value()`操作を使用して、このシーケンスの "次の値" を計算し

- 現在と次の値を、埋め込みディメンションの2倍のベクトルに連結します。`splice()`

次に、埋め込みレイヤの直後にこの関数を `Sequential()` のリストに挿入します。

In [16]:
# Your task: Add lookahead
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])
    
# Enable these when done:
#do_train()
#do_test()

### Task 3: Bidirectional Recurrent Model

Aha, knowledge of future words help. So instead of a one-word lookahead,
why not look ahead until all the way to the end of the sentence, through a backward recurrence?
Let us create a bidirectional model!

Your task is to implement a new layer that
performs both a forward and a backward recursion over the data, and
concatenates the output vectors.

Note, however, that this differs from the previous task in that
the bidirectional layer contains learnable model parameters.
In function-composition style,
the pattern to implement a layer with model parameters is to write a *factory function*
that creates a *function object*.

A function object, also known as [*functor*](https://en.wikipedia.org/wiki/Function_object), is an object that is both a function and an object.
Which means nothing else that it contains data yet still can be invoked as if it was a function.

For example, `Dense(outDim)` is a factory function that returns a function object that contains
a weight matrix `W`, a bias `b`, and another function to compute 
`input @ W + b.` (This is using 
[Python 3.5 notation for matrix multiplication](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-465).
In Numpy syntax it is `input.dot(W) + b`).
E.g. saying `Dense(1024)` will create this function object, which can then be used
like any other function, also immediately: `Dense(1024)(x)`. 

Let's look at an example for further clarity: Let us implement a new layer that combines
a linear layer with a subsequent batch normalization. 
To allow function composition, the layer needs to be realized as a factory function,
which could look like this:

```python
def DenseLayerWithBN(dim):
    F = Dense(dim)
    G = BatchNormalization()
    x = placeholder()
    apply_x = G(F(x))
    return apply_x
```

Invoking this factory function will create `F`, `G`, `x`, and `apply_x`. In this example, `F` and `G` are function objects themselves, and `apply_x` is the function to be applied to the data.
Thus, e.g. calling `DenseLayerWithBN(1024)` will
create an object containing a linear-layer function object called `F`, a batch-normalization function object `G`,
and `apply_x` which is the function that implements the actual operation of this layer
using `F` and `G`. It will then return `apply_x`. To the outside, `apply_x` looks and behaves
like a function. Under the hood, however, `apply_x` retains access to its specific instances of `F` and `G`.

Now back to our task at hand. You will now need to create a factory function,
very much like the example above.
You shall create a factory function
that creates two recurrent layer instances (one forward, one backward), and then defines an `apply_x` function
which applies both layer instances to the same `x` and concatenate the two results.

Alright, give it a try! To know how to realize a backward recursion in CNTK,
please take a hint from how the forward recursion is done.
Please also do the following:
* remove the one-word lookahead you added in the previous task, which we aim to replace; and
* make sure each LSTM is using `hidden_dim//2` outputs to keep the total number of model parameters limited.

_unchecked_

### タスク 3: 双方向リカレントモデル

なるほど、将来の言葉の知識に役立つ。なので1語の先読みではなく、文の最後までずっと前を向いて、後ろ向きな再発をスルーしてはいかがでしょうか?
私たちは双方向のモデルを作成してみましょう!

タスクは、データに対して前方と後方の両方の再帰を実行し、出力ベクタを連結する新しいレイヤーを実装することです。

ただし、双方向レイヤに可能モデルパラメータが含まれている点で、これは前のタスクとは異なることに注意してください。
関数合成スタイルでは、モデルパラメータを使用してレイヤを実装するパターンは、 *function オブジェクト*を作成する*ファクトリ関数*を記述することです。

関数オブジェクト ([None](https://en.wikipedia.org/wiki/Function_object)とも呼ばれます) は、関数とオブジェクトの両方であるオブジェクトです。
これは、それがまだそれが関数であるかのように呼び出すことができるデータが含まれていることを意味します。

たとえば、 `Dense(outDim)` は、ウェイトマトリックス、バイアス、およびを計算する別の関数を含む関数オブジェクトを返すファクトリ関数です `W` `b` `input @ W + b.` (これは、[行列乗算用の Python 3.5 表記法](https://docs.python.org/3/whatsnew/3.5.html#whatsnew-pep-465)を使用しています。
Numpy 構文では `input.dot(W) + b` です)。
例えば、 `Dense(1024)` は、この関数オブジェクトを作成し、その後、他の関数と同様に使用することもできます: `Dense(1024)(x)` 。

さらに明確にするための例を見てみましょう: 私たちは、後続のバッチ正規化とリニアレイヤーを組み合わせた新しい層を実装してみましょう。
関数の構成を許可するには、レイヤーをファクトリ関数として実現する必要があり、次のようになります。

`python
def DenseLayerWithBN(dim):
    F = Dense(dim)
    G = BatchNormalization()
    x = placeholder()
    apply_x = G(F(x))
    return apply_x`

このファクトリ関数を呼び出すと、 `F` 、 `G` 、 `x` 、および `apply_x` が作成されます。この例では、 `F` と `G` は関数オブジェクト自体であり、 `apply_x` はデータに適用される関数です。
したがって、例えば、 `DenseLayerWithBN(1024)` を呼び出すと、 `F` 、バッチ正規化関数オブジェクト `G` 、および `F` と `G` を使用して、この層の実際の動作を実装する関数である `apply_x` と呼ばれるリニアレイヤ関数オブジェクトを含むオブジェクトを作成します。その後、 `apply_x` を返します。外側には、 `apply_x` は、機能のように見えると動作します。しかし、ボンネットの下では、 `apply_x` は `F` と `G` の特定のインスタンスへのアクセスを保持します。

今戻って私たちの仕事に手を。これで、上記の例のように、ファクトリ関数を作成する必要があります。
2つの反復層インスタンスを作成するファクトリ関数 (1 つ前方、1つ後方) を作成し、両方のレイヤーインスタンスを同じ `x` に適用し、2つの結果を連結する `apply_x` 関数を定義します。

申し分なく、それを試してみてください!CNTK における後方再帰の実現方法を知るには、前方再帰の実行方法からヒントを得てください。
次の操作も実行してください:*前のタスクで追加した1つの単語の先読みを削除*します。各 LSTM が出力を使用して、 `hidden_dim//2` モデルパラメータの合計数を制限していることを確認します。

In [17]:
# Your task: Add bidirectional recurrence
def create_model():
    with C.layers.default_options(initial_state=0.1):  
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)
        ])

# Enable these when done:
#do_train()
#do_test()

The bidirectional model has 40% less parameters than the lookahead one. However, if you go back and look closely
you may find that the lookahead one trained about 30% faster.
This is because the lookahead model has both less horizontal dependencies (one instead of two
recurrences) and larger matrix products, and can thus achieve higher parallelism.

_unchecked_

双方向モデルは、先読み1よりも 40% 少ないパラメータを持っています。しかし、あなたが戻って、密接に見える場合は、先読みの1つは約 30% 速く訓練を見つけることができます。
これは、先読みモデルには、水平方向の依存関係 (2 つの繰り返しの代わりに1つ) と大規模なマトリックス製品の両方があるため、より高い並列処理を実現できます。

**Solution 1: Adding Batch Normalization**

_unchecked_

**解決策 1: バッチ正規化の追加**

In [18]:
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            #C.layers.BatchNormalization(), #Remove this comment if running on GPU
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            #C.layers.BatchNormalization(), #Remove this comment if running on GPU
            C.layers.Dense(num_labels)
        ])

do_train()
do_test()

Training 721479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.740198 * 18010, metric = 28.02% * 18010 4.223s (4264.7 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.665177 * 18051, metric = 14.30% * 18051 3.917s (4608.4 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.526256 * 17941, metric = 11.34% * 17941 3.898s (4602.6 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.395405 * 18059, metric = 8.22% * 18059 4.061s (4446.9 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.293512 * 17957, metric = 6.20% * 17957 3.996s (4493.7 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.264932 * 18021, metric = 5.73% * 18021 3.931s (4584.3 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.217258 * 17980, metric = 4.69% * 17980 3.941s (4562.3 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.209614 * 18025, metric =

**Solution 2: Add a Lookahead**

_unchecked_

**解決策 2: 先読みを追加する**

In [19]:
def OneWordLookahead():
    x = C.placeholder()
    apply_x = C.splice(x, C.sequence.future_value(x))
    return apply_x

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            OneWordLookahead(),
            C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_labels)        
        ])

do_train()
do_test()

Training 901479 parameters in 6 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.618925 * 18010, metric = 26.40% * 18010 4.567s (3943.5 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.572762 * 18051, metric = 12.46% * 18051 5.560s (3246.6 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.420728 * 17941, metric = 8.57% * 17941 5.254s (3414.7 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.297996 * 18059, metric = 6.28% * 18059 5.697s (3169.9 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.224015 * 17957, metric = 4.81% * 17957 5.612s (3199.8 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.207126 * 18021, metric = 4.61% * 18021 5.487s (3284.3 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.170268 * 17980, metric = 3.69% * 17980 5.538s (3246.7 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.164910 * 18025, metric = 

**Solution 3: Bidirectional Recurrent Model**

_unchecked_

**解決策 3: 双方向リカレントモデル**

In [20]:
def BiRecurrence(fwd, bwd):
    F = C.layers.Recurrence(fwd)
    G = C.layers.Recurrence(bwd, go_backwards=True)
    x = C.placeholder()
    apply_x = C.splice(F(x), G(x))
    return apply_x 

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim),
            BiRecurrence(C.layers.LSTM(hidden_dim//2), 
                                  C.layers.LSTM(hidden_dim//2)),
            C.layers.Dense(num_labels)
        ])

do_train()
do_test()

Training 541479 parameters in 9 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 10]: [Training] loss = 1.886776 * 18010, metric = 30.06% * 18010 7.619s (2363.8 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 0.683211 * 18051, metric = 14.83% * 18051 7.325s (2464.3 samples/s);
Finished Epoch[3 of 10]: [Training] loss = 0.521379 * 17941, metric = 11.42% * 17941 7.265s (2469.5 samples/s);
Finished Epoch[4 of 10]: [Training] loss = 0.394698 * 18059, metric = 8.11% * 18059 7.567s (2386.5 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 10]: [Training] loss = 0.288926 * 17957, metric = 6.06% * 17957 7.386s (2431.2 samples/s);
Finished Epoch[6 of 10]: [Training] loss = 0.267000 * 18021, metric = 5.73% * 18021 7.401s (2434.9 samples/s);
Finished Epoch[7 of 10]: [Training] loss = 0.215379 * 17980, metric = 4.69% * 17980 7.269s (2473.5 samples/s);
Finished Epoch[8 of 10]: [Training] loss = 0.206970 * 18025, metric =

## Task overview: Sequence classification

We will reuse the same data for this task. We revisit a sample again:

    19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O

The task of the neural network is to look at the query (column `S0`) and predict the
intent of the sequence (column `S1`). We will ignore the slot-tags (column `S2`) this time.


### Model Creation

The model we will use is a recurrent model consisting of an embedding layer,
a recurrent LSTM cell, and a dense layer to compute the posterior probabilities. Though very similar to the slot tagging model in this case we look only at the embedding from the last layer:


    intent                                                "flight"
                                                              ^          
                                                              |          
                                                          +-------+  
                                                          | Dense |   ...
                                                          +-------+  
                                                              ^         
                                                              |          
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"

Or, as a CNTK network description. Please have a quick look and match it with the description above:
(descriptions of these functions can be found at: [the layers reference](http://cntk.ai/pythondocs/layerref.html))

#### Points to note:
- The first difference between this model with the previous one is with regards to the specification of the label `y`. Since there is only one label per sequence, we use `C.input_variable`.
- The second difference is the use of [Stabilizer](http://ieeexplore.ieee.org/document/7472719/). We stabilize the embedded output. The stabilizer adds an additional scalar parameter to the learning that can help our network converge more quickly during training. 
- The third difference is the use of a layer function called `Fold`. As shown in the model above we want the model to have LSTM recurrence except the final one, we set up an LSTM recurrence. The final recurrence will be a Fold operation where we pick the hidden state from the last LSTM block and use it for classification of the entire sequence.


_unchecked_

## タスクの概要: シーケンス分類

このタスクでは、同じデータを再利用します。我々は再びサンプルを再訪:

    `19  |S0 178:1 |# BOS      |S1 14:1 |# flight  |S2 128:1 |# O
    19  |S0 770:1 |# show                         |S2 128:1 |# O
    19  |S0 429:1 |# flights                      |S2 128:1 |# O
    19  |S0 444:1 |# from                         |S2 128:1 |# O
    19  |S0 272:1 |# burbank                      |S2 48:1  |# B-fromloc.city_name
    19  |S0 851:1 |# to                           |S2 128:1 |# O
    19  |S0 789:1 |# st.                          |S2 78:1  |# B-toloc.city_name
    19  |S0 564:1 |# louis                        |S2 125:1 |# I-toloc.city_name
    19  |S0 654:1 |# on                           |S2 128:1 |# O
    19  |S0 601:1 |# monday                       |S2 26:1  |# B-depart_date.day_name
    19  |S0 179:1 |# EOS                          |S2 128:1 |# O
    `

ニューラルネットワークのタスクは、クエリ (列 `S0` ) を見て、シーケンス (列 `S1` ) の意図を予測することです。今回は、スロットタグ (カラム `S2` ) を無視します。

### モデルの作成

我々が使用するモデルは、後方の確率を計算するために、埋め込み層、再発 LSTM 細胞、および密な層から成る再発モデルです。私たちは最後の層からの埋め込みでのみ見て、このケースではスロットタグモデルに非常に似ていますが:

    `intent                                                "flight"
                                                              ^          
                                                              |          
                                                          +-------+  
                                                          | Dense |   ...
                                                          +-------+  
                                                              ^         
                                                              |          
              +------+   +------+   +------+   +------+   +------+   
         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->...
              +------+   +------+   +------+   +------+   +------+   
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
              +-------+  +-------+  +-------+  +-------+  +-------+
              | Embed |  | Embed |  | Embed |  | Embed |  | Embed |  ...
              +-------+  +-------+  +-------+  +-------+  +-------+
                  ^          ^          ^          ^          ^
                  |          |          |          |          |
    w      ------>+--------->+--------->+--------->+--------->+------... 
                 BOS      "show"    "flights"    "from"   "burbank"
    `

または、CNTK ネットワークの説明として。してください簡単に見て、上記の説明と一致する: (これらの関数の説明で見つけることができます:[レイヤー参照](http://cntk.ai/pythondocs/layerref.html))

#### 注意点:

- 前のものとのこのモデルの最初の相違はラベル `y` の指定に関してある。シーケンスごとにラベルが1つしかないため、 `C.input_variable` を使用します。

- 2つ目の違いは、[スタビライザー](http://ieeexplore.ieee.org/document/7472719/)の使用です。埋め込み出力を安定化します。安定剤は、トレーニング中に我々のネットワークがより迅速に収束することができます学習に追加のスカラーパラメータを追加します。

- 3番目の違いは、 `Fold` と呼ばれるレイヤ関数の使用です。上のモデルに示されているように、我々は最終的なものを除いて LSTM 再発を持っているモデルにしたい, 我々は LSTM 再発を設定する.最終的な再発は、我々は最後の LSTM ブロックから非表示の状態を選択し、全体のシーケンスの分類のためにそれを使用してフォールド操作になります。

In [21]:
# number of words in vocab, slot labels, and intent labels
vocab_size = 943 ; num_intents = 26    

# model dimensions
emb_dim    = 150
hidden_dim = 300

# Create the containers for input feature (x) and the label (y)
x = C.sequence.input_variable(vocab_size)
y = C.input_variable(num_intents)

def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Stabilizer(),
            C.layers.Fold(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_intents, name='classify')
        ])

We create the `criterion` function with the new model and correspondingly update the placeholder. 

_unchecked_

新しいモデルを使用して `criterion` 関数を作成し、それに応じてプレースホルダを更新します。

In [22]:
criterion = create_criterion_function(create_model())
criterion.replace_placeholders({criterion.placeholders[0]: C.sequence.input_variable(num_intents)})

Composite(Combine): Input('Input14623', [#, *], [26]), Placeholder('labels', [???], [???]) -> Output('Block14593_Output_0', [#], [1]), Output('Block14613_Output_0', [#], [])

The same train code can be used except for the fact that in this case we provide the `intent` tags as the labels.

_unchecked_

同じ列車コードはこの場合私達がラベルとして `intent` の札を提供するという事実を除いて使用することができる。

In [23]:
def do_train():
    global z
    z = create_model()
    reader = create_reader(data['train']['file'], is_training=True)
    train(reader, z, 5, 'intent')
do_train()

Training 690477 parameters in 7 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 5]: [Training] loss = 0.585308 * 18004, metric = 14.42% * 18004 79.421s (226.7 samples/s);
Finished Epoch[2 of 5]: [Training] loss = 0.153843 * 17998, metric = 3.92% * 17998 74.394s (241.9 samples/s);
Finished Epoch[3 of 5]: [Training] loss = 0.081964 * 18000, metric = 2.17% * 18000 59.060s (304.8 samples/s);
Finished Epoch[4 of 5]: [Training] loss = 0.069163 * 18000, metric = 1.92% * 18000 58.621s (307.1 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 5]: [Training] loss = 0.018180 * 17998, metric = 0.47% * 17998 58.345s (308.5 samples/s);


Now we can measure the model accuracy by going through all the examples in the test set and using the C.eval.Evaluator method. 

_unchecked_

ここでは、テストセットのすべての例と、c. eval エバリュエータメソッドを使用して、モデルの精度を測定できます。

In [24]:
def do_test():
    reader = create_reader(data['test']['file'], is_training=False)
    evaluate(reader, z, 'intent')
do_test()
z.classify.b.value

Finished Evaluation [1]: Minibatch[1-23]: metric = 0.00% * 893;


array([ 0.51183742, -0.29180217, -0.41856512, -0.28958377, -0.72160971,
       -0.24883319,  0.02916328, -0.0852626 ,  0.19014142, -0.41936243,
       -0.23639332, -0.29029393, -0.74258387, -0.12296562,  0.34665295,
       -0.46388549, -0.73981428, -0.2296015 , -0.78151304, -0.24418215,
       -0.52702737,  0.10101102, -0.27433836, -0.24181543, -0.39551306,
        0.30569023], dtype=float32)

The following block of code illustrates how to evaluate a single sequence. Additionally, we show how one can pass in the information using NumPy arrays.

_unchecked_

次のコードブロックは、1つのシーケンスを評価する方法を示しています。さらに、NumPy 配列を使用して情報をどのように渡すことができるかを示します。

In [25]:
# load dictionaries
query_wl = [line.rstrip('\n') for line in open(data['query']['file'])]
intent_wl = [line.rstrip('\n') for line in open(data['intent']['file'])]
query_dict = {query_wl[i]:i for i in range(len(query_wl))}
intent_dict = {intent_wl[i]:i for i in range(len(intent_wl))}

# let's run a sequence through
seq = 'BOS flights from new york to seattle EOS'
w = [query_dict[w] for w in seq.split()] # convert to word indices
onehot = np.zeros([len(w),len(query_dict)], np.float32)
for t in range(len(w)):
    onehot[t,w[t]] = 1

pred = z(x).eval({x:[onehot]})[0]
best = np.argmax(pred)
print(best)
print(seq, ":", intent_wl[best])

14
BOS flights from new york to seattle EOS : flight


**Task 4: Use all hidden states for sequence classification**

In the last model, we looked at the output of the last LSTM block. There is another way to model, where we aggregate the output from all the LSTM blocks and use the aggregated output to the final `Dense` layer.

So your task will be to replace the `C.layers.Fold` with `C.layers.Recurrence` layer function. This is the explicit way of setting up recurrence. You will aggregate all the intermediate outputs from LSTM blocks using `C.sequence.reduce_sum`. Note: this is different from last model where we looked only at the output of the last LSTM block. 

So please go ahead and modify the configuration and see what happens.

If everything went right, you will notice improved accuracy (`metric`). The solution is presented right after, but we suggest you refrain from looking at the solution.

_unchecked_

**タスク 4: シーケンス分類にすべての非表示状態を使用する**

最後のモデルでは、最後の LSTM ブロックの出力を調べました。我々はすべての LSTM ブロックからの出力を集計し、最終的な `Dense` 層に集約された出力を使用して、モデル化する別の方法があります。

だからあなたのタスクは、 `C.layers.Recurrence` 層関数と `C.layers.Fold` を置き換えることになります。これは、繰り返しを設定する明示的な方法です。`C.sequence.reduce_sum`を使用して、LSTM ブロックからすべての中間出力を集計します。注: これは最後のモデルとは異なり、最後の LSTM ブロックの出力だけを見ました。

だから先に行って、設定を変更し、何が起こるかを参照してください。

すべてが正しかった場合は、改善された精度 ( `metric` ) に気づくでしょう。ソリューションは、直後に提示されますが、我々はあなたがソリューションを見てから控えることをお勧めします。

In [26]:
# Replace the line with Fold operation
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Stabilizer(),
            C.layers.Fold(C.layers.LSTM(hidden_dim), go_backwards=False),
            C.layers.Dense(num_intents, name='classify')
        ])
# Enable these when done:
#do_train()
#do_test()

Aggregating all the intermediate states improves the accuracy for the same number of iterations without any significant increase in the computation time.

_unchecked_

すべての中間状態を集約すると、計算時間が大幅に増加することなく、同じ数のイテレーションの精度が向上します。

**Solution 4: Use all hidden states for sequence classification**

_unchecked_

**解決策 4: シーケンス分類にすべての非表示状態を使用する**

In [27]:
def create_model():
    with C.layers.default_options(initial_state=0.1):
        return C.layers.Sequential([
            C.layers.Embedding(emb_dim, name='embed'),
            C.layers.Stabilizer(),
            C.sequence.reduce_sum(C.layers.Recurrence(C.layers.LSTM(hidden_dim), go_backwards=False)),
            C.layers.Dense(num_intents, name='classify')
        ])
    
do_train()
do_test()

Training 690477 parameters in 7 parameter tensors.
Learning rate per minibatch: 0.020999999999999998
Finished Epoch[1 of 5]: [Training] loss = 0.193482 * 18004, metric = 4.07% * 18004 56.904s (316.4 samples/s);
Finished Epoch[2 of 5]: [Training] loss = 0.028983 * 17998, metric = 0.64% * 17998 73.820s (243.8 samples/s);
Finished Epoch[3 of 5]: [Training] loss = 0.059269 * 18000, metric = 1.24% * 18000 71.494s (251.8 samples/s);
Finished Epoch[4 of 5]: [Training] loss = 0.005993 * 18000, metric = 0.18% * 18000 61.856s (291.0 samples/s);
Learning rate per minibatch: 0.010499999999999999
Finished Epoch[5 of 5]: [Training] loss = 0.000344 * 17998, metric = 0.01% * 17998 57.114s (315.1 samples/s);
Finished Evaluation [1]: Minibatch[1-23]: metric = 0.00% * 893;
